In [1]:
import os
import json
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import random
import math
import pulp
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model
import networkx as nx
import concurrent.futures
import threading
from multiprocessing import Pool
from copy import deepcopy
import gurobipy as gp
from prettytable import PrettyTable
import plotly.graph_objects as go
import matplotlib.colors as mcolors
import colorsys
import numba
from matplotlib.colors import ListedColormap, BoundaryNorm

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
load C:\Users\MWX1248700\VScodeProjects\mcf_spine_leaf\py_env\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\MWX1248700\VScodeProjects\mcf_spine_leaf\py_env\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\MWX1248700\VScodeProjects\mcf_spine_leaf\py_env\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\MWX1248700\VScodeProjects\mcf_spine_leaf\py_env\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\MWX1248700\VScodeProjects\mcf_spine_leaf\py_env\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\MWX1248700\VSc

In [2]:
alpha_list = [
    "-1.0",
    "-0.9",
    "-0.8",
    "-0.7",
    "-0.6",
    "-0.5",
    "-0.4",
    "-0.3",
    "-0.2",
    "-0.1",
    "0.0",
    "0.1",
    "0.2",
    "0.3",
    "0.4",
    "0.5",
    "0.6",
    "0.7",
    "0.8",
    "0.9",
    "1.0",
]

In [38]:
for entry in os.listdir('new_special_multitests_scenario2'):
    print(entry)

2_16_8
3_16_8
7_16_8


In [28]:
class Test:
    def __init__(self, t, l, s, ranking, layer_colors, communication_colors, final_ranking, result):
        self.tasks = t
        self.leaves = l
        self.spines = s
        self.ranking = ranking
        self.layer_colors = layer_colors
        self.communication_colors = communication_colors
        self.final_ranking = final_ranking
        self.result = result
    

In [4]:
def parse_coloring(colors_list):
    res = {}
    for (task, stage, rank_from, rank_to), size in colors_list:
        if task not in res:
            res[task] = {}
        if stage not in res[task]:
            res[task][stage] = []
        res[task][stage].append([rank_from, rank_to, size])
    return res
def read_test(name, test_dir_name, result_dir_name):
    tasks, leaves, spines = list(map(int, name.split('_')))
    
    res_tests = []
    test_dir_path = f'{test_dir_name}/{name}'
    res_dir_path = f'{result_dir_name}/{name}'
    count = int(len([entry for entry in os.listdir(test_dir_path) if os.path.isfile(os.path.join(test_dir_path, entry))])/2)
    for num in range(count):
        test_data = None
        test_color = None

        res_coloring = None
        res_ranking = None
        res_result = None
        with open(os.path.join(test_dir_path, f"test_{num}.json"), 'r') as f:
            test_data = json.load(f)
        with open(os.path.join(test_dir_path, f"color_{num}.json"), 'r') as f:
            test_color = json.load(f)
        
        with open(os.path.join(res_dir_path, f"coloring_{num}.json"), 'r') as f:
            res_coloring = json.load(f)
        with open(os.path.join(res_dir_path, f"ranking_{num}.json"), 'r') as f:
            res_ranking = json.load(f)
        with open(os.path.join(res_dir_path, f"test_{num}.json"), 'r') as f:
            res_result = json.load(f)
            
        res_tests += [ Test(tasks, leaves, spines, test_data[i], test_color[i], parse_coloring(res_coloring[i]), res_ranking[i], res_result[i]) for i in range(len(test_color)) ]
    return res_tests

In [5]:
tests = read_test('7_16_8', 'new_special_multitests', 'new_special_multitests_scenario2')

In [6]:
print([t.result for t in tests])

[1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]


In [7]:
tests[2].result

0

In [8]:
def get_ranking_graph(t, l, s, ranking):
    G = nx.Graph()
    leaves_host_number_on_tasks = [
        [(task, ranking[task][leaf]) for task in range(t)] for leaf in range(l)
    ]
    for leaf in leaves_host_number_on_tasks:
        for task_id, size in leaf:
            for i in range(1, size + 1):
                for j in range(i + 1, size + 1):
                    G.add_edge(f"{task_id}_{i}", f"{task_id}_{j}")
        for from_ in range(0, len(leaf)):
            for to_ in range(from_ + 1, len(leaf)):
                task_from, size_from = leaf[from_]
                task_to, size_to = leaf[to_]
                for i in range(1, size_from + 1):
                    for j in range(1, size_to + 1):
                        G.add_edge(f"{task_from}_{i}", f"{task_to}_{j}")
    return G

In [9]:
i = 2
t,l,s = tests[i].tasks, tests[i].leaves, tests[i].spines
print(t,l,s)
tests[i].final_ranking, tests[i].ranking

7 16 8


([[[0, 1, 2, 3, 4],
   [5, 6, 7],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   []],
  [[0, 1, 2],
   [3, 4, 5],
   [6, 7],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   []],
  [[],
   [6, 7],
   [1, 2, 3, 5],
   [0, 4],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   []],
  [[],
   [],
   [1, 3],
   [4, 6],
   [0, 2, 5],
   [7],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   []],
  [[],
   [],
   [],
   [4, 11, 14, 21],
   [7, 13, 20, 24, 29],
   [0, 1, 2, 5, 18, 26, 27],
   [6, 9, 12, 16, 22],
   [15, 19],
   [3, 8, 10, 25, 30],
   [17, 28],
   [23, 31],
   [],
   [],
   [],
   [],
   []],
  [[],
   [],
   [],
   [],
   [],
   [],
   [14, 16, 17],
   [3, 4, 7, 9, 19, 30],
   [21, 25, 31],
   [0, 6, 11, 15, 28, 29],
   [1, 2, 10, 13, 22, 26],
   [5, 8, 12, 20, 24, 27],
   [23],
   [18],
   [],
   []],
  [[],
   [],
   [],
   [],
   [],
   [],
   [

In [10]:
G = nx.MultiGraph()
keys = G.add_edges_from([(1,2), (1,3), (2,3), (1,3), (1,2)])
for u, v, k in G.edges(keys=True):
    print(u, v, k)
G[1][2][1]['color'] = 2

for to in G.neighbors(1):
    print(G[1][to])

1 2 0
1 2 1
1 3 0
1 3 1
2 3 0
{0: {}, 1: {'color': 2}}
{0: {}, 1: {}}


In [11]:
d = {1:2, 2:3, 3:-1}
min(d.items(), key=lambda x:x[1])

(3, -1)

In [24]:
def finish_broken_test(test:Test):
    additional_colors = []
    
    for task in range(test.tasks):
        hosts = sum(test.ranking[task])

        host_to_leaf = [-1] * hosts
        for leaf_id, leaf in enumerate(test.final_ranking[task]):
            for host in leaf:
                host_to_leaf[host] = leaf_id

        stages = int(np.log2(hosts))
        for stage in range(stages):
            edge_dict = {}
            
            add_power = 2**stage
            G = nx.MultiGraph()
            for rank_from in range(hosts):
                rank_to = rank_from ^ add_power
                if host_to_leaf[rank_from] == host_to_leaf[rank_to]:
                    continue
                edge_dict[(rank_from, rank_to)] = G.add_edge(f'{host_to_leaf[rank_from]}_s', f'{host_to_leaf[rank_to]}_d', rank_s=rank_from, rank_d=rank_to, color=-1)
            
            if not (task in test.communication_colors and stage in test.communication_colors[task]):
                continue
            for rank_from, rank_to, size in test.communication_colors[task][stage]:
                G[f'{host_to_leaf[rank_from]}_s'][f'{host_to_leaf[rank_to]}_d'][edge_dict[(rank_from, rank_to)]]['color'] = size
                
            for rank_from in range(hosts):
                rank_to = rank_from ^ add_power
                if host_to_leaf[rank_from] == host_to_leaf[rank_to]:
                    continue
                leaf_s = f'{host_to_leaf[rank_from]}_s'
                leaf_d = f'{host_to_leaf[rank_to]}_d'
                key = edge_dict[(rank_from, rank_to)]
                if G[leaf_s][leaf_d][key]['color'] != -1:
                    continue
                available_colors = min(test.ranking[task][host_to_leaf[rank_from]], test.ranking[task][host_to_leaf[rank_to]])
                used_colors = {c:0 for c in range(1, available_colors+1)}

                for leaf in [leaf_s, leaf_d]:
                    for leaf_nbr in G.neighbors(leaf):
                        for edge_key, color_dict in G[leaf][leaf_nbr].items():
                            if color_dict['color'] != -1 and color_dict['color'] <= available_colors:
                                used_colors[color_dict['color']] += 1
                min_color, min_times = min(used_colors.items(), key=lambda x:x[1])
                G[leaf_s][leaf_d][key]['color'] = min_color
                additional_colors.append([[task, stage, rank_from, rank_to], min_color])
    return additional_colors

In [29]:
# print(finish_broken_test(tests[1]))
# print(tests[1].final_ranking[2])
tests[1].layer_colors[2][1]

{'0_1': 0,
 '0_2': 1,
 '0_3': 2,
 '0_4': 3,
 '0_5': 4,
 '0_6': 5,
 '0_7': 6,
 '1_1': 7,
 '1_2': 1,
 '1_3': 2,
 '1_4': 3,
 '1_5': 4,
 '1_6': 5,
 '1_7': 6,
 '2_1': 0,
 '2_2': 1,
 '2_3': 2,
 '2_4': 3,
 '2_5': 4,
 '2_6': 5,
 '3_1': 6,
 '3_2': 7,
 '3_3': 2,
 '3_4': 3,
 '4_1': 4,
 '4_2': 5,
 '4_3': 0,
 '4_4': 1,
 '4_5': 2,
 '4_6': 3,
 '4_7': 6,
 '4_8': 7,
 '5_1': 7,
 '5_2': 0,
 '5_3': 1,
 '5_4': 2,
 '5_5': 3,
 '5_6': 4,
 '5_7': 5,
 '5_8': 6,
 '6_1': 5,
 '6_2': 6,
 '6_3': 3,
 '6_4': 4,
 '6_5': 0,
 '6_6': 1,
 '6_7': 2,
 '6_8': 7}

In [26]:
def update_broken_test(test:Test):
    additional_colors = finish_broken_test(test)
    for (task, stage, rank_from, rank_to), size in additional_colors:
        if task not in test.communication_colors:
            test.communication_colors[task] = {}
        if stage not in test.communication_colors[task]:
            test.communication_colors[task][stage] = []
        test.communication_colors[task][stage].append([rank_from, rank_to, size])

In [27]:
for test in tests:
    if test.result != 0:
        continue
    update_broken_test(test)
    

In [34]:
def get_coloring(test:Test):
    res = []
    for task, task_dict in test.communication_colors.items():
        for stage, vec in task_dict.items():
            for (rank_from, rank_to, size) in vec:
                res.append([[task, stage, rank_from, rank_to], test.layer_colors[f'{task}_{size}']])
    return res

In [37]:
colorings = [get_coloring(test) for test in tests]
colorings[0]

[[[0, 0, 2, 3], 2],
 [[0, 0, 3, 2], 2],
 [[0, 1, 1, 3], 2],
 [[0, 1, 3, 1], 2],
 [[0, 2, 0, 4], 0],
 [[0, 2, 1, 5], 1],
 [[0, 2, 2, 6], 2],
 [[0, 2, 4, 0], 0],
 [[0, 2, 5, 1], 1],
 [[0, 2, 6, 2], 2],
 [[1, 0, 4, 5], 5],
 [[1, 0, 5, 4], 5],
 [[1, 1, 4, 6], 5],
 [[1, 1, 6, 4], 5],
 [[1, 2, 1, 5], 3],
 [[1, 2, 2, 6], 4],
 [[1, 2, 3, 7], 5],
 [[1, 2, 5, 1], 3],
 [[1, 2, 6, 2], 4],
 [[1, 2, 7, 3], 5],
 [[2, 0, 4, 5], 2],
 [[2, 0, 5, 4], 2],
 [[2, 1, 4, 6], 2],
 [[2, 1, 6, 4], 2],
 [[2, 2, 1, 5], 0],
 [[2, 2, 2, 6], 1],
 [[2, 2, 3, 7], 2],
 [[2, 2, 5, 1], 0],
 [[2, 2, 6, 2], 1],
 [[2, 2, 7, 3], 2],
 [[3, 0, 2, 3], 7],
 [[3, 0, 3, 2], 7],
 [[3, 1, 1, 3], 7],
 [[3, 1, 3, 1], 7],
 [[3, 2, 0, 4], 5],
 [[3, 2, 1, 5], 6],
 [[3, 2, 2, 6], 7],
 [[3, 2, 4, 0], 5],
 [[3, 2, 5, 1], 6],
 [[3, 2, 6, 2], 7],
 [[4, 0, 0, 1], 3],
 [[4, 0, 1, 0], 3],
 [[4, 0, 2, 3], 1],
 [[4, 0, 3, 2], 0],
 [[4, 0, 6, 7], 4],
 [[4, 0, 7, 6], 4],
 [[4, 0, 8, 9], 6],
 [[4, 0, 9, 8], 6],
 [[4, 0, 10, 11], 0],
 [[4, 0, 11, 10], 

In [156]:
additional_colors = finish_broken_test(tests[3])
print([x for x in additional_colors if x[0][0] == 1 and x[0][1] == 1],'\n')
print(tests[3].final_ranking,'\n')
print(tests[3].ranking,'\n')
tests[3].communication_colors[1][1]

[[[1, 1, 4, 6], 1], [[1, 1, 6, 4], 1], [[1, 1, 13, 15], 2], [[1, 1, 15, 13], 2], [[1, 1, 25, 27], 3], [[1, 1, 27, 25], 3]] 

[[[1, 2, 4, 5, 10, 12, 15, 20], [6, 7, 19, 23, 25, 28], [0, 3, 8, 16, 18, 21, 26, 29], [14, 17, 24, 27, 30], [9, 11, 13, 22], [31], [], [], [], [], [], [], [], [], [], []], [[], [1, 7], [], [10, 18, 30], [2, 8, 31], [5, 14, 19, 21, 23, 28, 29], [0, 4, 12, 13, 16, 17, 20, 25], [9, 22, 24], [], [11, 26], [], [6, 15, 27], [3], [], [], []], [[], [], [], [], [36], [], [], [1, 2, 4, 5, 16], [7, 8, 9, 11, 19, 31, 56, 57], [14, 17, 18, 21, 28, 49], [15, 22, 23, 24, 25, 26, 32, 39], [10, 27, 29, 30, 60], [12, 20, 33, 35, 37, 38, 40], [34, 41, 42, 43, 44, 46, 47, 62], [13, 48, 50, 51, 52, 53, 54, 55], [0, 3, 6, 45, 58, 59, 61, 63]]] 

[[8, 6, 8, 5, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 3, 3, 7, 8, 3, 0, 2, 0, 3, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 5, 8, 6, 8, 5, 7, 8, 8, 8]] 



[[0, 2, 1],
 [1, 3, 1],
 [2, 0, 2],
 [3, 1, 1],
 [5, 7, 2],
 [7, 5, 2],
 [8, 10, 3],
 [9, 11, 1],
 [10, 8, 2],
 [11, 9, 1],
 [12, 14, 6],
 [14, 12, 6],
 [16, 18, 2],
 [17, 19, 7],
 [18, 16, 1],
 [19, 17, 7],
 [20, 22, 3],
 [22, 20, 3],
 [24, 26, 2],
 [26, 24, 2],
 [28, 30, 1],
 [29, 31, 3],
 [30, 28, 3],
 [31, 29, 1]]